In [1]:
from sqlalchemy import create_engine, inspect, text, MetaData, select

: 

In [2]:
# database connection details
username = "postgres"
password = "1234"
host = "localhost"
post = 5432
database = "postgres"

# table_name = "btc_data"

# connect to the database
engine = create_engine(f"postgresql://{username}:{password}@{host}:{post}/{database}")

: 

In [4]:
import requests, json
from datetime import datetime

def get_market_data(
    ticker: str = "XXBTZUSD"
):
    
    # Daily interval
    interval = str(60*24)

    # Construct the API URL for fetching OHLC data
    url = f"https://api.kraken.com/0/public/OHLC?pair={ticker}&interval={interval}"

    # Set up request headers
    headers = {'Accept': 'application/json'}

    try:
        # Make the request to the Kraken API
        response = requests.request(
            "GET", url, headers=headers,
            data={}, timeout=10
        )
    except requests.exceptions.Timeout as e:
        # Print timeout exception message
        print(e)

    # Parse the JSON response
    data = json.loads(response.text)['result'][ticker]
    
    return data

In [5]:
ticker_pair = "XXBTZUSD"
data = get_market_data(ticker_pair)

In [6]:
data

[[1675728000,
  '22765.5',
  '23347.7',
  '22750.1',
  '23253.2',
  '23055.8',
  '2505.20814770',
  25138],
 [1675814400,
  '23253.3',
  '23450.0',
  '22675.5',
  '22964.8',
  '23049.1',
  '2330.96864109',
  30058],
 [1675900800,
  '22963.6',
  '23008.2',
  '21700.0',
  '21792.0',
  '22333.1',
  '4427.29733421',
  39050],
 [1675987200,
  '21792.0',
  '21939.9',
  '21475.0',
  '21639.8',
  '21716.1',
  '2769.51193674',
  27382],
 [1676073600,
  '21638.2',
  '21909.8',
  '21610.9',
  '21868.2',
  '21780.6',
  '1349.09720346',
  14538],
 [1676160000,
  '21868.2',
  '22099.9',
  '21640.1',
  '21786.0',
  '21898.7',
  '1108.48249854',
  17674],
 [1676246400,
  '21786.0',
  '21902.2',
  '21359.1',
  '21791.2',
  '21637.4',
  '2505.14047810',
  27721],
 [1676332800,
  '21790.5',
  '22331.3',
  '21505.0',
  '22213.1',
  '21978.1',
  '2688.79461358',
  24782],
 [1676419200,
  '22213.2',
  '24366.4',
  '22038.5',
  '24334.5',
  '23266.0',
  '5601.61561807',
  41639],
 [1676505600,
  '24340.9',
 

In [7]:
import pandas as pd

# convert the data to a pandas dataframe
def convert_api_call_to_df(data):
    df = pd.DataFrame(
        data,
        columns=[
            'date', 'open',
            'high', 'low',
            'close', 'vmap',
            'volume', 'count'
        ]
    )

    # convert to numeric
    df = df.apply(pd.to_numeric, errors='coerce')
    # convert to datetime
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    return df

In [8]:
df = convert_api_call_to_df(data)

In [9]:
# select subset of columns
def select_columns(df, columns):
    return df.loc[:, columns]

In [10]:
df = select_columns(df, ['date', 'open', 'high', 'low', 'close'])

In [11]:
df

,date,open,high,low,close
0,2023-02-07,22765.5,23347.7,22750.1,23253.2
1,2023-02-08,23253.3,23450.0,22675.5,22964.8
2,2023-02-09,22963.6,23008.2,21700.0,21792.0
3,2023-02-10,21792.0,21939.9,21475.0,21639.8
4,2023-02-11,21638.2,21909.8,21610.9,21868.2
...,...,...,...,...,...
715,2025-01-22,106162.8,106293.4,103271.3,103648.5
716,2025-01-23,103648.6,106833.2,101281.7,103920.2
717,2025-01-24,103920.2,107080.6,102800.0,104840.4
718,2025-01-25,104840.4,105194.1,104250.0,104705.2


In [13]:
from great_expectations.dataset import PandasDataset
from great_expectations.data_context import DataContext

# create a PandasDataset
df_ge = PandasDataset(df)

# set up expectation parameters
dtypes = ["datetime64[ns]", "float64", "float64", "float64", "float64"]
cols = ["date", "open", "high", "low", "close"]

# create the expectations for each column
for column, dtype in zip(cols, dtypes):
    # dtype check
    df_ge.expect_column_values_to_be_of_type(column=column, type_=dtype)
    if column != "date":
        # null check
        df_ge.expect_column_values_to_not_be_null(column=column)

# unique dates check
df_ge.expect_column_values_to_be_unique(column="date")

# run the validation
validation_results = df_ge.validate()

# check if all expectations are met
all_success = all(result["success"] for result in validation_results["results"])

# raise an exception if validation fails
if all_success:
    print(f"Validation passed. {len(validation_results['results'])} expectations were met.")
else:
    # write validation results to a file
    failed_results = [result for result in validation_results["results"] if not result["success"]]
    with open("btc_data_validation_results.json", "w") as f:
        json.dump(failed_results, f, indent=2)
    raise Exception("Validation failed. Check validation_results.json for details.")



Validation passed. 10 expectations were met.


In [14]:
# calculate exponential moving average on a column
def apply_ema_7(df, column_name, window_size):
    new_key = f"{column_name}_ema_{window_size}"
    
    # apply exponential moving average to a column
    df[new_key] = (
        df[column_name]
        .ewm(span=window_size,
             adjust=False)
        .mean()
        .round(2)
    )
    return df

In [15]:
df = apply_ema_7(df, "close", 7)

df

,date,open,high,low,close,close_ema_7
0,2023-02-07,22765.5,23347.7,22750.1,23253.2,23253.20
1,2023-02-08,23253.3,23450.0,22675.5,22964.8,23181.10
2,2023-02-09,22963.6,23008.2,21700.0,21792.0,22833.82
3,2023-02-10,21792.0,21939.9,21475.0,21639.8,22535.32
4,2023-02-11,21638.2,21909.8,21610.9,21868.2,22368.54
...,...,...,...,...,...,...
715,2025-01-22,106162.8,106293.4,103271.3,103648.5,102640.66
716,2025-01-23,103648.6,106833.2,101281.7,103920.2,102960.54
717,2025-01-24,103920.2,107080.6,102800.0,104840.4,103430.51
718,2025-01-25,104840.4,105194.1,104250.0,104705.2,103749.18


In [16]:
# insert into postgresql database

create_query = text(
                    """
                        CREATE TABLE IF NOT EXISTS btc_data (
                        date DATE PRIMARY KEY,
                        open FLOAT,
                        high FLOAT,
                        low FLOAT,
                        close FLOAT,
                        close_ema_7 FLOAT
                        );
                    """
                )

insert_query = text(
                    """
                        INSERT INTO btc_data (date, open, high, low, close, close_ema_7)
                        VALUES (:date, :open, :high, :low, :close, :close_ema_7)
                        ON CONFLICT (date) DO NOTHING
                    """
                )

with engine.connect() as conn:
    conn.execute(create_query)
    conn.commit()


with engine.connect() as conn:
    for _, row in df.iterrows():
        conn.execute(
            insert_query,
            {
                "date": row["date"],
                "open": row["open"],
                "high": row["high"],
                "low": row["low"],
                "close": row["close"],
                "close_ema_7": row["close_ema_7"]
            }
        )
        conn.commit()

In [17]:
# retrieve the close and close ema_7 data

query = text(
             """
                SELECT date, close, close_ema_7
                FROM btc_data
             """
)

with engine.connect() as conn:
    result = conn.execute(query)
    data = result.fetchall()



In [18]:
import plotly.express as px

# create a plot of the close and close ema 7 data
def show_close_ema(data):
    df = pd.DataFrame(data, columns=["date", "close", "close_ema_7"])
    fig = px.line(df, x='date', y=['close', 'close_ema_7'], title=f"{ticker_pair}_price_data")
    fig.show()
    fig.write_html("btc_close_ema.html")

In [19]:
show_close_ema(data)